In [27]:
import torch

batch_size = 5
nb_digits = 10
# Dummy input that HAS to be 2D for the scatter (you can use view(-1,1) if needed)
y = torch.LongTensor(batch_size,1).random_() % nb_digits
# One hot encoding buffer that you create out of the loop and just keep reusing
y_onehot = torch.FloatTensor(batch_size, nb_digits)

# In your for loop
y_onehot.zero_()
y_onehot.scatter_(1, y, 1)

print(y)
print(y_onehot)

tensor([[9],
        [4],
        [1],
        [7],
        [6]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])


In [39]:
batch_size = 2
seq_len = 4
num_classes = 10

token_ids = torch.LongTensor(batch_size,seq_len).random_() % nb_digits



In [40]:
token_ids

tensor([[6, 0, 9, 0],
        [7, 5, 8, 1]])

In [46]:

onehots = torch.zeros(batch_size, seq_len, num_classes)
onehots.scatter_(dim=2, index=token_ids.unsqueeze(2), value=1)

tensor([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [1]:
import torch

In [2]:
torch.manual_seed(1)

In [3]:
x=torch.zeros(2,3)
for i in range(2):
    for j in range(3):
        x[i,j] = j

In [4]:
y=torch.zeros(2,3)
for i in range(2):
    for j in range(3):
        y[i,j] = j

In [5]:
w0 = torch.ones(3,3)
for i in range(3):
    for j in range(3):
        w0[i,j] = j
w0 = torch.nn.Parameter(w0)

In [6]:
x

tensor([[0., 1., 2.],
        [0., 1., 2.]])

In [7]:
w1 = torch.ones(3,3)
for i in range(3):
    for j in range(3):
        w1[i,j] = j
w1 = torch.nn.Parameter(w1)

w2 = torch.ones(3,3)
for i in range(3):
    for j in range(3):
        w2[i,j] = j
w2 = torch.nn.Parameter(w2)

w3 = torch.ones(3,3)
for i in range(3):
    for j in range(3):
        w3[i,j] = j + 2
w3 = torch.nn.Parameter(w3)

In [8]:
l1=None
l2=None

In [9]:
g = w3
adv1 = w1
adv2 = w2

In [10]:
sgd = torch.optim.SGD((g,adv1,adv2), lr=0.5)

In [11]:
print(g)
print(adv1)
print(adv2)

Parameter containing:
tensor([[2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.]], requires_grad=True)
Parameter containing:
tensor([[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]], requires_grad=True)
Parameter containing:
tensor([[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]], requires_grad=True)


# Compute grads for generator 

In [12]:
print(g.grad)
print(adv1.grad)

None
None


In [13]:
sgd.zero_grad()

In [14]:
print(g.grad)
print(adv1.grad)

None
None


---

In [15]:
adv1.requires_grad=False # HAVE TO BE CALLED BEFORE FORWARD!

---

In [16]:
tmp = x.matmul(w3) 
l1 = tmp.matmul(w1).sum()

In [17]:
print(g.grad)
print(adv1.grad)

None
None


In [18]:
#adv1.grad.data.zero_()

In [19]:
print(g.grad)
print(adv1.grad)

None
None


# Compute grads for discriminator

In [20]:
w1.requires_grad=True

In [21]:
tmp2=tmp.detach() 
l2 = tmp2.matmul(w1).sum() # might be differnet from l1

In [22]:
l3 = l1+l2
l3.backward()

In [23]:
print(g.grad)
print(adv1.grad)

tensor([[ 0.,  0.,  0.],
        [ 6.,  6.,  6.],
        [12., 12., 12.]])
tensor([[12., 12., 12.],
        [18., 18., 18.],
        [24., 24., 24.]])


In [24]:
print(g.grad)
print(adv1.grad)

tensor([[ 0.,  0.,  0.],
        [ 6.,  6.,  6.],
        [12., 12., 12.]])
tensor([[12., 12., 12.],
        [18., 18., 18.],
        [24., 24., 24.]])


In [25]:
sgd.step()

In [26]:
print(g)
print(adv1)
print(adv2)

Parameter containing:
tensor([[ 2.,  3.,  4.],
        [-1.,  0.,  1.],
        [-4., -3., -2.]], requires_grad=True)
Parameter containing:
tensor([[ -6.,  -5.,  -4.],
        [ -9.,  -8.,  -7.],
        [-12., -11., -10.]], requires_grad=True)
Parameter containing:
tensor([[0., 1., 2.],
        [0., 1., 2.],
        [0., 1., 2.]], requires_grad=True)
